Chuẩn đo lường mà bạn chọn để đánh giá thuật toán trong machine learning thì **rất quan trọng**. Lựa chọn _chuẩn đo lường_ chi phối cái cách biểu diễn thuật toán machine learning ở đo lường và so sánh.<br>
Chúng chi phối cái cách bạn weight sự quan trọng của các đặc trưng khác nhau trong kết quả và sự lựa chọn cuối cùng với thuật toán để chọn.

## Algorithm Evaluation Metrics

Chú ý là khi dùng hàm cross_val_score để làm báo cáo. Thì nó cho phép được sử dụng nhiều _điểm chuẩn đo lường_ (different scoring metrics) khác nhau nhưng tất cả điểm có thể bị xếp theo từ lớn đến nhỏ (điểm lớn thì tốt nhất). Một vài chuẩn đo lường (như mean squared error) thì được xếp từ nhỏ đến lớn (điểm nhỏ thì tốt nhất) và theo như vậy sẽ là số âm. Đây là một chú ý quan trọng bởi một vài chuẩn điểm sẽ bị báo cáo là **số âm** trong khi theo _định nghĩa_ thì **không thể nào** là **số âm** được.

## I. Classification Metrics

Có vô số chuẩn đo lường được sử dụng trong problem này. Trong phần này ta sẽ học sử dụng những chuẩn đo lường sau:
- Classification Accuracy
- Logarithmic Loss
- Area Under ROC Curve
- Confusion Matrix
- Classification Report

### 1.1 Classification Accuracy

Classification accuracy là tỷ lệ số tiên đoán đúng trên tổng số tiên đoán. Đây chuẩn đo lường đánh giá phổ biến nhất trong bài toán classification, nó cũng là thứ bị sử dụng sai nhiều nhất.<br>
Nó thật sự chỉ phù hợp khi có một số lượng đồng đều quan sát trên từng class và tất cả các tiên đoán và tiên đoán sai đều đồng quan trọng.<br>
Thường thì không phải cho trường hợp này. Bên dưới là ví dụ tính toán classification accuracy.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LogisticRegression
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv('pima-indians-diabetes.csv', names=names)
array = dataframe.values
X = array[:, :8] # input 
Y = array[:, 8] # output
dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
num_folds = 10
seed= 7
kfold = KFold(n_splits=num_folds, random_state=seed)
model = LogisticRegression(solver='liblinear')
scoring = 'accuracy' # dùng chuẩn đo lường Accuracy
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print("Accuracy: %.3f%% (%.3f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 76.951% (4.841%)


Tỷ lệ chính xác xấp xỉ 77% với độ lệch 4.8%

### 1.2 Logarithmic Loss

Logarithmic loss (hay logloss) là một biểu diễn chuẩn đo lường cho sự tiên đoán xác suất thành viên đến 1 class được cho. Biến xác suất scalar giữa 0 và 1 có thể được xem như sự đo lường sự tự tin của một tiên đoán bởi 1 thuật toán.<br>
Tiên đoán đúng hoặc sai thì được thưởng hoặc phạt tương ứng với độ tự tin của tiên đoán đó.

In [16]:
scoring = 'neg_log_loss' # Dùng chuẩn đo lường log loss
results = cross_val_score(model, X, Y, scoring=scoring, cv =kfold)
print('Logloss: %.3f (%.3f)' % (results.mean(), results.std()))

Logloss: -0.493 (0.047)


Logloss có giá trị nhỏ thì tốt hơn, với Logloss bằng 0 là hoàn hảo. Như đã đề cập ở trên, đo lường bị đảo thành từ nhỏ đến lớn khi sử dụng hàm cross_val_score()

### 1.3 Area Under ROC Curve

Area under ROC Curve (hay AUC) chuẩn đo lường biểu diễn cho bài toán binary classification (phân loại nhị phân). AUC đại diện cho khả năng của mô hình phân biệt positive và negative của các class.<br>
Vùng 1.0 đại diện cho mô hình có tất cả tiên đoán là hoàn hảo. Vùng 0.5 đại diện cho mô hình tốt một cách ngẫu nhiên.<br>
Bài toán phân loại nhị phân là sự đánh đổi giữa sensitivity và specificity.
- Sensitivity là tỷ lệ true positive hay còn gọi là recall. Nó là số lượng thực thể từ positive class (first class) thật sự được tiên đoán chính xác.
- Specificity còn gọi là tỷ lệ true negative. Nó là số lượng thực thể từ negative class (second class) mà thực sự được tiên đoán chính xác.

In [18]:
scoring = 'roc_auc'
results = cross_val_score(model, X, Y, scoring=scoring, cv =kfold)
print('AUC: %.3f (%.3f)' % (results.mean(), results.std()))

AUC: 0.824 (0.041)


AUC gần bằng 1 và lớn hơn 0.5

### 1.4 Confusion Matrix

Confusion matrix là một đại diện hữu ích cho độ chính xác của mô hình với 2 hoặc hơn nhiều classes. Bảng đại diện cho sự tiên đoán trên trục x và độ chính xác của kết quả trên trục y.

In [3]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state =seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
matrix = confusion_matrix(Y_test, predicted)
print(matrix)

[[141  21]
 [ 41  51]]


Số lượng tiên đoán chính xác nằm ở đường chéo ma trận (diagnol line of matrix)

### 1.5 Classification Report

In [4]:
from sklearn.metrics import classification_report
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state =seed)
model = LogisticRegression(solver='liblinear')
model.fit(X_train, Y_train)
predicted = model.predict(X_test)
report = classification_report(Y_test, predicted)
print(report)

              precision    recall  f1-score   support

         0.0       0.77      0.87      0.82       162
         1.0       0.71      0.55      0.62        92

    accuracy                           0.76       254
   macro avg       0.74      0.71      0.72       254
weighted avg       0.75      0.76      0.75       254



Giải thích, ví dụ ở nhãn 1:
- Precision ở label 1 là 0.71 tức là trong 100 lần dự đoán là 1 thì mô hình dự đoán đúng 71 lần.
- Recall ở label 1 là 0.55 tức là thật sự có 100 label 1 thì mô hình chỉ tìm ra được 55 'con số' 1.
- f1-score là điểm điều hòa trung gian của Precision và Recall.

Nói thêm ở ví dụ trên:
- Precision thì biết tin cậy của lần dự đoán nhưng không biết được mô hình đã tìm được bao nhiêu 'con số' 1. Ví dụ precision là 1 tức là dự đoán đúng 100% nhưng nếu mô hình chỉ dự đoán có '1 con' đánh nhãn 1 trong khi có 100 con 1 thì mô hình đó là mô hình **tồi**.
- Recall thì biết khả năng tìm 'con số' 1 là bao nhiêu nhưng không biết đã lẫn bao nhiêu nhãn bị sai vào. Ví dụ recall là 1 tức là đã tìm được 100% nhãn 1 nhưng nếu lẫn trong đó đến 200 con là nhãn 0 thì đó là mô hình **tồi**.
- F1 là trung hòa giữa Precision và Recall
$$ F1 = 2\times\frac{Precision \times Recall}{Precision + Recall}$$

Tùy bài toán mà ta ưu tiên chọn Precision, Recall hay f1. Thường ta sẽ cố gắng tunning để Precision và Recall về gần với nhau.

## II. Regression Metrics

### 2.1 Mean Absolute Error

Mean Absolute Error hay MAE là tổng giá trị tuyệt đối sự khác nhau giữa giá trị tiên đoán và thực tế. Điều này cho ta hiểu việc tiên đoán của ta sai số cỡ nào. Vì thế chuẩn đo lường này cho biết được độ lớn của sai số nhưng không biết được hướng sai số (trên hay dưới của giá trị thực)

In [10]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression
names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
dataframe = pd.read_csv('housing.csv',delim_whitespace=True, names=names)
array = dataframe.values
X = array[:, 0:13] # input
Y = array[:, 13] # output
dataframe.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [11]:
num_folds = 10
seed = 7
kfold = KFold(n_splits=num_folds, random_state=seed)
model = LinearRegression()
scoring = 'neg_mean_absolute_error'
results = cross_val_score(model, X, Y, cv=kfold, scoring=scoring)
print('MAE %.3f (%.3f)' % (results.mean(), results.std()))

MAE -4.005 (2.084)


Giá trị mà bằng 0 tức là không có lỗi hay tiên đoán hoàn hảo. Như logloss, chuẩn đo lường này bị chuyển dấu vì hàm corss_val_score.

### 2.2 Mean Squared Error

Mean Squared Error hay MSE cũng giống như MAE, tức giúp ta biết về độ lớn của sai số. Sử dụng trung bình của bình phương sự khác nhau giữa giá trị dự đoán và giá trị thực tế. Biến các đơn vị về đơn vị chuẩn của biến output.

$$ MSE = \frac{1}{N}\times \sum_{i=1}^{n} (y-\hat{y})^2$$

In [12]:
scoring = 'neg_mean_squared_error'
results = cross_val_score(model, X, Y ,cv=kfold, scoring=scoring)
print('MSE: %.3f (%.3f)' % (results.mean(), results.std()))

MSE: -34.705 (45.574)


Chuẩn đo lường này cũng bị chuyển dấu vì hàm corss_val_score. Vì thế scoring đứng trước có chữ neg = negative.

### 2.3 $R^2$ Metric

Chuẩn đo lường $R^2$ chỉ ra bộ giá trị tiên đoán có khớp với các giá trị thực tế không. Nó là một giá trị giữa 0 và 1 trong đó 0 là không khớp và 1 là khớp hoàn toàn.

In [16]:
scoring = 'r2'
results = cross_val_score(model, X, Y ,cv=kfold, scoring=scoring)
print('R^2: %.3f (%.3f)' % (results.mean(), results.std()))

R^2: 0.203 (0.595)
